## Przygotowanie danych

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import ipywidgets as widgets
from IPython.display import display
from IPython.display import Image

In [ ]:
file = "ks_projects_201801.csv"
filepath = os.path.join(os.path.abspath(""), file)

In [ ]:
df = pd.read_csv(filepath)

In [ ]:
# Usunięcie wartości NaN
df = df.dropna()

# Ujednolicenie kolumn z datami, przekształcenie na format datetime
df['deadline_new'] = pd.to_datetime(df['deadline']).dt.date
df['launched_new'] = pd.to_datetime(df['launched']).dt.date

# Obliczenie czasu trwania projektów, oczyszczenie danych - przekształcenie na typ int
df['duration'] = df['deadline_new'] - df['launched_new']
df['duration'] = df['duration'].map(lambda x: int(str(x).split(" ")[0]))

# Wyodrębnienie dwóch stanów projektów: successful = 1, pozostałe = 0
df["state_new"] = df["state"].map(lambda x: 1 if x == "successful" else 0 )

# Usunięcie niepotrzebnych kolumn
to_drop = ["currency",
           "deadline",
           "goal",
           "launched",
           "pledged",
           "state",
           "usd pledged"]

df.drop(to_drop, inplace=True, axis=1)

# Połączenie Luksemburga, Belgii i Holandii w Beneluks

df["country"] = df["country"].replace(["LU", "BE"],"NL")

# Odrzucenie krajów, w których liczba projektów jest poniżej 1000

df = df[(df["country"] != "SG") 
        & (df["country"] != "JP") 
        & (df["country"] != "AT") 
        & (df["country"] != "HK")
        & (df["country"] != "NO")
        & (df["country"] != "CH")
        & (df["country"] != "IE")]

# Wyodrębnienie roku i miesiąca z danych 

df['year_launched'] = pd.DatetimeIndex(df['launched_new']).year
df['month_launched'] = pd.DatetimeIndex(df['launched_new']).month

# Usunięcie rekordów z 1970 roku

df = df[df["year_launched"] != 1970]

df.head()

## Wybór kraju

In [ ]:
choice_widget = widgets.Dropdown(
                    layout={'width': 'max-content'}, 
                    options=[("Australia",'AU'),("Benelux",'NL'),("Canada",'CA'),("Denmark",'DK'),("France",'FR'),("Germany",'DE'),("Great Britain",'GB'),("Itlay",'IT'),("Mexico",'MX'),("New Zealand",'NZ'),("Spain",'ES'),("Sweden",'SE'),("United States",'US')],
                    description='Select country:',
                    style={'description_width': 'initial'})

choice_widget

In [ ]:
cf = df[df['country']==choice_widget.value]
cf

## Czy użytkownik chce poznać szanse na sfinansowanie swojego projektu?

In [ ]:
yes_no_widget = widgets.ToggleButtons(
                    options=["Yes", "No"],
                    description="Do you want to know what are the chances for your project to succeed?",
                    disabled=False,
                    style={"description_width": "initial"},
                    button_style="info", # 'success', 'info', 'warning', 'danger' or ''
                    tooltips=["Yes, I do", "No, I don't"],
                )
yes_no_widget

## Wybór parametrów projektu przez użytkownika

### 1. Kategoria

In [ ]:
main_cat = list(cf["main_category"].unique())
main_cat_widget = widgets.Dropdown(
                    options=main_cat,
                    value=main_cat[0],
                    description="Main category in {}".format(choice_widget.value),
                    style={"description_width": "initial"},
                    disabled=False,
                )
main_cat_widget

### 2. Podkategoria

In [ ]:
main_cat_select = cf[cf["main_category"] == main_cat_widget.value]
cat = list(main_cat_select["category"].unique())

cat_widget = widgets.Dropdown(
                options=cat,
                value=cat[0],
                description="Category",
                style={"description_width": "initial"},
                disabled=False,
            )
cat_widget

### 3. Czas trwania zbiórki

In [ ]:
duration_widget = widgets.IntSlider(
                    value=0,
                    min=cf["duration"].min(),
                    max=cf["duration"].max(),
                    step=1,
                    description= "How many days is the fundraising going to last?",
                    disabled=False,
                    continuous_update=False,
                    orientation="horizontal",
#                     style={"description_width": "initial"},
#                     layout={'width': 'max-content'},
#                     readout=True,
#                     readout_format="d"
                )

duration_widget

### 4. Kwota zbiórki

In [ ]:
money_widget = widgets.FloatText(
                    value=10,
                    description="How much money do you want to collect?($)",
                    style={"description_width": "initial"},
                    layout={'width': 'max-content'},
                    disabled=False
                )

money_widget

## Oszacowanie prawdopodobieństwa, że projekt złożony w wybranej kategorii zostanie sfinansowany

In [ ]:
# Projekty, które zostały sfinansowane w wybranej kategorii

mcat_s = cf[(cf["main_category"] == main_cat_widget.value) & (cf["state_new"] == 1)]

# Projekty, które zostały sfinansowane w pozostałych kategoriach

no_mcat_s = cf[(cf["main_category"] != main_cat_widget.value) & (cf["state_new"] == 1)]

# Projekty, które NIE zostały sfinansowane w wybranej kategorii

mcat_no_s = cf[(cf["main_category"] == main_cat_widget.value) & (cf["state_new"] == 0)]

# Projekty, które NIE zostały sfinansowane w pozostałych kategoriach

no_mcat_no_s = cf[(cf["main_category"] != main_cat_widget.value) & (cf["state_new"] == 0)]

# Prawdopodobieństwo, że projekt uzyska finansowanie pod warunkiem, że został złożony w określonej kategorii

prob_main_category = 100 * mcat_s["state_new"].count() / (mcat_s["state_new"].count() + mcat_no_s["state_new"].count())
print(f"Probability of funding for main category {main_cat_widget.value}: {prob_main_category:.1f} %")

## Oszacowanie prawdopodobieństwa, że projekt złożony w danej podkategorii zostanie sfinansowany

In [ ]:
# Projekty, które zostały sfinansowane w wybranej podkategorii

cat_s = cf[(cf["category"] == cat_widget.value) & (cf["state_new"] == 1)]

# Projekty, które zostały sfinansowane w pozostałych podkategoriach

no_cat_s = cf[(cf["category"] != cat_widget.value) & (cf["state_new"] == 1)]

# Projekty, które NIE zostały sfinansowane w wybranej podkategorii

cat_no_s = cf[(cf["category"] == cat_widget.value) & (cf["state_new"] == 0)]

# Projekty, które NIE zostały sfinansowane w pozostałych podkategoriach

no_cat_no_s = cf[(cf["category"] != cat_widget.value) & (cf["state_new"] == 0)]

# Prawdopodobieństwo, że projekt uzyska finansowanie pod warunkiem, że został złożony w określonej podkategorii

prob_category = 100 * cat_s["state_new"].count() / (cat_s["state_new"].count() + cat_no_s["state_new"].count())
print(f"Probability of funding for category {cat_widget.value}: {prob_category:.1f} %")

## Oszacowanie prawdopodobieństwa uzyskania finansowania projektu w zależności od długości trwania zbiórki

In [ ]:
interval_list = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]


for start, stop in interval_list:

    # Sprawdzenie, w którym przedziale znajduje się wartość podana przez użytkownika
    if duration_widget.value > start and duration_widget.value <= stop:

        # Projekty, które zostały sfinansowane w wybranym przedziale czasowym
        dur_s = cf[ (cf["duration"] > start) & (cf["duration"] <= stop) & (cf["state_new"] == 1) ]

        # Projekty, które NIE zostały sfinansowane w wybranym przedziale czasowym
        dur_no_s = cf[ (cf["duration"] > start) & (cf["duration"] <= stop) & (cf["state_new"] == 0) ]

        # Prawdopodobieństwo, że projekt uzyska finansowanie pod warunkiem, że czas zbiórki jest w określonym przedziale
        prob_duration = 100 * dur_s["state_new"].count() / (dur_s["state_new"].count() + dur_no_s["state_new"].count())

        print(f"Probability of funding for project duration {duration_widget.value} days: {prob_duration:.1f} %")
    

## Oszacowanie prawdopodobieństwa uzyskania finansowania projektu w zależności od deklarowanej kwoty

In [ ]:
amount_list = list(cf["usd_goal_real"].quantile([0.2, 0.4, 0.6, 0.8]))
print(amount_list)

    
if money_widget.value < amount_list[0]:
        
    amount_s = cf[ (cf["usd_goal_real"] < amount_list[0]) & (cf["state_new"] == 1) ]
    amount_no_s = cf[ (cf["usd_goal_real"] < amount_list[0]) & (cf["state_new"] == 0) ]
    prob_amount = 100 * amount_s["state_new"].count() / (amount_s["state_new"].count() + amount_no_s["state_new"].count())
        
    print(f"Probability of funding for declared amount {money_widget.value} $: {prob_amount:.1f} %")
        
elif money_widget.value >= amount_list[0] and money_widget.value < amount_list[1]:
        
    amount_s = cf[ (cf["usd_goal_real"] >= amount_list[0]) & (cf["usd_goal_real"] < amount_list[1]) & (cf["state_new"] == 1)]
    amount_no_s = cf[ (cf["usd_goal_real"] >= amount_list[0]) & (cf["usd_goal_real"] < amount_list[1]) & (cf["state_new"] == 0) ]
    prob_amount = 100 * amount_s["state_new"].count() / (amount_s["state_new"].count() + amount_no_s["state_new"].count())
        
    print(f"Probability of funding for declared amount {money_widget.value} $: {prob_amount:.1f} %")
    
elif money_widget.value >= amount_list[1] and money_widget.value < amount_list[2]:
        
    amount_s = cf[ (cf["usd_goal_real"] >= amount_list[1]) & (cf["usd_goal_real"] < amount_list[2]) & (cf["state_new"] == 1)]
    amount_no_s = cf[ (cf["usd_goal_real"] >= amount_list[1]) & (cf["usd_goal_real"] < amount_list[2]) & (cf["state_new"] == 0) ]
    prob_amount = 100 * amount_s["state_new"].count() / (amount_s["state_new"].count() + amount_no_s["state_new"].count())
        
    print(f"Probability of funding for declared amount {money_widget.value} $: {prob_amount:.1f} %")
    
elif money_widget.value >= amount_list[2] and money_widget.value < amount_list[3]:
        
    amount_s = cf[ (cf["usd_goal_real"] >= amount_list[2]) & (cf["usd_goal_real"] < amount_list[3]) & (cf["state_new"] == 1)]
    amount_no_s = cf[ (cf["usd_goal_real"] >= amount_list[2]) & (cf["usd_goal_real"] < amount_list[3]) & (cf["state_new"] == 0) ]
    prob_amount = 100 * amount_s["state_new"].count() / (amount_s["state_new"].count() + amount_no_s["state_new"].count())
        
    print(f"Probability of funding for declared amount {money_widget.value} $: {prob_amount:.1f} %")
        
elif money_widget.value >= amount_list[3]:
        
    amount_s = cf[(cf["usd_goal_real"] >= amount_list[3]) & (cf["state_new"] == 1)]
    amount_no_s = cf[(cf["usd_goal_real"] >= amount_list[3]) & (cf["state_new"] == 0) ]
    prob_amount = 100 * amount_s["state_new"].count() / (amount_s["state_new"].count() + amount_no_s["state_new"].count())
        
    print(f"Probability of funding for declared amount {money_widget.value} $: {prob_amount:.1f} %")

## Oszacowanie całkowitego prawdopodobieństwa uzyskania finansowania (reguła łańcuchowa prawdopodobieństwa)

In [ ]:
def total_prob():
    interval_list = [(0, 10), (10, 20), (20, 30), (30, 40), (40, 50), (50, 60), (60, 70), (70, 80), (80, 90), (90, 100)]

    for start, stop in interval_list:
        if duration_widget.value > start and duration_widget.value <= stop:
            a = start
            b = stop
        
    amount_list = list(cf["usd_goal_real"].quantile([0.2, 0.4, 0.6, 0.8]))
   
    if money_widget.value < amount_list[0]:
        c = 0
        d = amount_list[0]
    elif money_widget.value >= amount_list[0] and money_widget.value < amount_list[1]:
        c = amount_list[0]
        d = amount_list[1]
    elif money_widget.value >= amount_list[1] and money_widget.value < amount_list[2]:
        c = amount_list[1]
        d = amount_list[2]
    elif money_widget.value >= amount_list[2] and money_widget.value < amount_list[3]:
        c = amount_list[2]
        d = amount_list[3]  
    elif money_widget.value >= amount_list[3]:
        c = amount_list[3]
        d = 10000000000000000 


    p_abcde = np.sum((cf["main_category"] == main_cat_widget.value)
                     & (cf["category"] == cat_widget.value) 
                     & (cf["state_new"] == 1)
                     & (cf["duration"] > a) & (cf["duration"] <= b)
                     & (cf["usd_goal_real"] > c) & (cf["usd_goal_real"] <= d)) / cf["ID"].count()

    p_abcd = np.sum((cf["main_category"] == main_cat_widget.value) 
                     & (cf["category"] == cat_widget.value) 
                     & (cf["duration"] > a) & (cf["duration"] <= b)
                     & (cf["usd_goal_real"] > c) & (cf["usd_goal_real"] <= d)) / cf["ID"].count()

    if p_abcde == 0 or p_abcd == 0:
        return(f"The probability cannot be determined")
    else:
        p = (p_abcde / p_abcd) * 100
        return(f"Probability of getting fund is {p:.1f} %")

## Odpowiedź dla użytkownika

In [ ]:
if yes_no_widget.value == "Yes":
    print(f"Probability of funding for main category {main_cat_widget.value}: {prob_main_category:.1f} %")
    print()
    print(f"Probability of funding for category {cat_widget.value}: {prob_category:.1f} %")
    print()
    print(f"Probability of funding for project duration {duration_widget.value} days: {prob_duration:.1f} %")
    print()
    print(f"Probability of funding for declared amount {money_widget.value} $: {prob_amount:.1f} %")
    print()
    print(total_prob())